# Import

In [1]:
from elasticsearch import Elasticsearch
import os
from elasticsearch_dsl import Search
import pandas as pd
import json
file_path = 'C:\\Users\\AM\\Desktop\\9.1\\books_info.csv'

df = pd.read_csv(file_path)

print(len(df))

42174


# Count Null

In [2]:
null_counts = df.isnull().sum()
print(null_counts)

title                  0
url                    0
contributors        6555
date                  88
format                 0
fulltext_url           0
trove_id               0
language           14284
rights             14409
pages                  0
form                  16
volume             40751
parent             40751
children           41934
text_downloaded        0
text_file          12522
dtype: int64


# Connect To Elasticsearch

In [3]:
es = Elasticsearch(['http://127.0.0.1:9200'])

# Create Index

In [4]:
url_create_index = 'http://localhost:9200/books'

response_create_index = requests.put(url_create_index)
print(response_create_index.text)

{"acknowledged":true,"shards_acknowledged":true,"index":"yas"}


# Show Indexs

In [5]:
import requests
url = 'http://localhost:9200/_cat/indices'

response = requests.get(url)
print(response.text)

yellow open ball          VortP4azSyeqY7Lh5s3YNQ 1 1     80 0    8kb    8kb
yellow open books         YJdTchFbQFeRbV4n2nqPEg 1 1 500009 0   61mb   61mb
yellow open test          J2YjS9iFSyu35qa3HvId1g 1 1     20 0 68.4kb 68.4kb
yellow open tickets-index -NjiCuziQuqPZB_CH1_qSg 1 1     11 0  5.6kb  5.6kb
yellow open links         O7WsyVAYSwacnkhfWeL1Tw 1 1    135 8 93.8kb 93.8kb
yellow open yas           QaNpuPkPSd2W3fpIJektkg 1 1      6 0  5.3kb  5.3kb
yellow open tiex          BwGz2XDOSASK8nXcwixGUw 1 1     11 0  5.6kb  5.6kb



# Insert Inf Books To Elasticsearch

In [6]:
o = 0
index_name = 'books'
ld =[]

for index, row in df.iterrows():
    date =[]
    
    forma = row['format'].split("|")
    if not pd.isnull(row['date']):
        date = row['date'].split("-")
       
    for i in range(len(forma)):
        if len(date) == 1:
            o +=1
            data ={
                "_index": index_name,
                "_id": o,
                "_source":{
                    "title":  row['title'],
                    "url": row['url'],
                    "contributors": row['contributors'],
                    "date" : date[0],
                    "format" : forma[i],
                    "fulltext_url" : row['fulltext_url'],
                    "trove_id" : row['trove_id'],
                    "language" : row['language'],
                    "rights" : row['rights'],
                    "pages" : int(row['pages']),
                    "form" : row['form'],
                    "volume" : row['volume'],
                    "parent" : row['parent'],
                    "children" : row['children'],
                    "text_downloaded" : row['text_downloaded'],
                    "text_file" : row['text_file']
                }
            }
            ld.append(data)
    
        elif len(date)>1:           
            for k in range(int(date[0]),int(date[1])+1):
                o +=1
                data = {
                    "_index": index_name,
                    "_id": o,
                    "_source":{
                        "title":  row['title'],
                        "url": row['url'],
                        "contributors": row['contributors'],
                        "date" : k,
                        "format" : forma[i],
                        "fulltext_url" : row['fulltext_url'],
                        "trove_id" : row['trove_id'],
                        "language" : row['language'],
                        "rights" : row['rights'],
                        "pages" : int(row['pages']),
                        "form" : row['form'],
                        "volume" : row['volume'],
                        "parent" : row['parent'],
                        "children" : row['children'],
                        "text_downloaded" : row['text_downloaded'],
                        "text_file" : row['text_file']
                    }
                }           
                ld.append(data)
            
        else:
            o +=1
            data = {
                "_index": index_name,
                "_id": o,
                "_source":{
                  "title":  row['title'],
                "url": row['url'],
                "contributors": row['contributors'],
                "date" : '',
                "format" : forma[i],
                "fulltext_url" : row['fulltext_url'],
                "trove_id" : row['trove_id'],
                "language" : row['language'],
                "rights" : row['rights'],
                "pages" : int(row['pages']),
                "form" : row['form'],
                "volume" : row['volume'],
                "parent" : row['parent'],
                "children" : row['children'],
                "text_downloaded" : row['text_downloaded'],
                "text_file" : row['text_file']  
                } 
            }
            ld.append(data)
            



import math
for document in ld:
    for key, value in document["_source"].items():
        if isinstance(value, float) and math.isnan(value):
            document["_source"][key] = None 

print(o)
print(len(ld))

500009
500009


In [47]:
import time

start_time = time.time()
helpers.bulk(es, ld)
end_time = time.time()

execution_time = end_time - start_time
print(f"Time Insert: {execution_time} Seconds")

Time Insert: 111.93666315078735 Seconds


# Querys

### Query 1

## این کوئری در محیط کوئری کیبانا اجرا شد

### Query 2

## این کوئری در محیط کوئری کیبانا اجرا شد

<br><br><br>

# Level 2

## این قسمت نمودارهای کیبانا است

<br><br><br>

# Level 3

In [35]:
from cassandra.cluster import Cluster
import time
from elasticsearch import Elasticsearch

# Connect Cassandra

In [58]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('ate')

# Create Table Parent

In [100]:
session.execute("CREATE TABLE inf_parent (trove_id text,namebook text,aut text,text_downloaded text,formats text, \
pages int ,date_p int, PRIMARY KEY (trove_id))")

# Create Table Children

In [91]:
session.execute("CREATE TABLE inf_children (trove_id text,namebook text,aut text,text_downloaded text,formats text, \
pages int ,date_p int, PRIMARY KEY (trove_id))")

# Get Data With Qurey - Insert To inf_children

In [96]:
es = Elasticsearch(hosts=['localhost'], port=9200)

from cassandra.query import BatchStatement
batch = BatchStatement()
insert_query = session.prepare(
        "INSERT INTO inf_children (trove_id, namebook, aut, text_downloaded, formats, pages, date_p) \
        VALUES (?, ?, ?, ?, ?, ?, ?)")

query = {
    "query": {
        "exists": {
            "field": "parent"
        }
    },
    "size": 200
}

scroll_time = "1m"
result = es.search(index="books", body=query, scroll=scroll_time)
scroll_id = result["_scroll_id"]
total_results = result["hits"]["total"]["value"]
all_results = []

while len(all_results) < total_results:
    hits = result["hits"]["hits"]
    all_results.extend(hits)
    result = es.scroll(scroll_id=scroll_id, scroll=scroll_time)
    scroll_id = result["_scroll_id"]
    batch.clear()
    batch = BatchStatement()
    for data in hits:
        data = data['_source']
        try:
            d = int(data['date'])
        except ValueError:
            d = 0
        batch.add(insert_query, (data['trove_id'], data['title'], data['contributors'],data['url'],data['format'], data['pages'], d))
    session.execute(batch)

<ipython-input-96-bd3197bacfd3>:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="books", body=query, scroll=scroll_time)


In [99]:
r = session.execute('SELECT COUNT(*) FROM inf_children')
print(r.one()[0])

1419


# Get Data With Qurey - Insert To inf_parent

In [103]:
es = Elasticsearch(hosts=['localhost'], port=9200)

from cassandra.query import BatchStatement
batch = BatchStatement()
insert_query = session.prepare(
        "INSERT INTO inf_parent (trove_id, namebook, aut, text_downloaded, formats, pages, date_p) \
        VALUES (?, ?, ?, ?, ?, ?, ?)")

query = {
    "query": {
        "exists": {
            "field": "children"
        }
    },
    "size": 200
}

scroll_time = "1m"
result = es.search(index="books", body=query, scroll=scroll_time)
scroll_id = result["_scroll_id"]
total_results = result["hits"]["total"]["value"]
all_results = []

while len(all_results) < total_results:
    hits = result["hits"]["hits"]
    all_results.extend(hits)
    result = es.scroll(scroll_id=scroll_id, scroll=scroll_time)
    scroll_id = result["_scroll_id"]
    batch.clear()
    batch = BatchStatement()
    for data in hits:
        data = data['_source']
        try:
            d = int(data['date'])
        except ValueError:
            d = 0
        batch.add(insert_query, (data['trove_id'], data['title'], data['contributors'],data['url'],data['format'], data['pages'], d))
    session.execute(batch)

<ipython-input-103-f8de3ad0e7d8>:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="books", body=query, scroll=scroll_time)


In [104]:
r = session.execute('SELECT COUNT(*) FROM inf_parent')
print(r.one()[0])

238


# Level 4

## Q1

In [108]:
result_set_parent = session.execute("SELECT AVG(pages) FROM inf_parent")
result_set_children = session.execute("SELECT AVG(pages) FROM inf_children")
average_parent = result_set_parent.one()[0]
average_children = result_set_children.one()[0]
print('Average Parent: ',average_parent)
print('Average Children: ',average_children)

Average Parent:  0
Average Children:  298


## Q2

In [120]:
result_set = session.execute("SELECT * FROM inf_children WHERE date_p < 2000 ALLOW FILTERING")
for row in result_set:
    print(row.namebook, row.date_p)

Kangxi zi dian 1716
Kangxi zi dian 1716
Zhong xiao jie yi er du mei zhuan 1800
Kangxi zi dian 1716
Kangxi zi dian 1716
Kangxi zi dian 1716
Kangxi zi dian 1716
Kangxi zi dian 1716
Bankoku zusetsu fūdo bussanshi / Seiyōjin senjutsu yakuhon 1800
Shi ming shu zheng / Liu Xi zhuan ; Bi Yüan shu zheng 1800
Five years on : implementation of the Commonwealth Government responses to the recommendations of the Royal Commission into Aboriginal Deaths in Custody 1997
Sōmoku zusetsu. Inuma Yokusai cho 1907
Quan di wu da zhou nü su tong kao / by Young J. Allen ; assisted by Yin Pao-Lu 1903
Kangxi zi dian 1716
The cyclopedia of South Australia in two volumes : an historical and commercial review, descriptive and biographical, facts, figures, and illustrations: an epitome of progress / edited by H.T. Burgess 1978
Kangxi zi dian 1716
Kangxi zi dian 1716
How justice is administered in the Northern Territory of South Australia 1886
Volume PETHpam 1842 1872
Kangxi zi dian 1716
Bankoku zusetsu fūdo b

In [121]:
result_set = session.execute("SELECT * FROM inf_parent WHERE date_p < 2000 ALLOW FILTERING")
for row in result_set:
    print(row.namebook, row.date_p)

Lists of men-of-war, 1650-1700 1938
[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career] 1889
Report of the proceedings of the Queensland Government schooner "Spitfire" in search of the mouth of the River Burdekin, on the north-eastern coast of Australia : and of the exploration of a portion of that coast extending from Gloucester Island to Halifax Bay 1860
Westminster Abbey : a service of memorial and thanksgiving for the Right Honourable Harold Edward Holt, C.H., M.P., Prime Minister of the Commonwealth of Australia ; born 5 August 1908, died 17 December 1967 ; Thursday 18th January, 1968 at 12 noon 1968
Brief sketch of the life of Theophilus Gum, the prophet, author and teacher of the new dispensation : with a criticism of his extraordinary writings / by the author of Popular preachers of our day 1901
Yokohama tenpozu 1800
Be men : a sermon preached in the St. Kilda Congregational Church, on Sunday morning, 11th April, 1858 / by Thomas B